In [2]:
import numpy as np
import pandas as pd

In [3]:
 import os
 for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
         print(os.path.join(dirname, filename))

In [4]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [7]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/19 11:27:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tutorial').getOrCreate()

In [9]:
df_train = spark.read.csv('/Users/payel/Downloads/train.csv',header = True,inferSchema=True)

In [10]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [11]:
type(df_train)

pyspark.sql.dataframe.DataFrame

In [12]:
df_test = spark.read.csv('/Users/payel/Downloads/test.csv',header=True,inferSchema=True)

In [13]:
type(df_test)

pyspark.sql.dataframe.DataFrame

In [14]:
df_test.show()

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|          330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|    7.0| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|          240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|          315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|         3101298|12.2875| null|       S|
|        897|     3|Svensson, Mr. Joh...|  male|14.0|    0|    0|            7538|  9.225| null|       S|
|        898|     3|Connolly, Miss. Kate|femal

In [15]:
df_train.count()

891

In [16]:
df_test.count()

418

In [17]:
891+418

1309

In [18]:
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [19]:
df_test.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [20]:
# As the Survived column is missing in test we will create a dummy column and put a value of -1 in it

In [21]:
from pyspark.sql.functions import lit
df_test = df_test.withColumn('Survived', lit(-1))

In [22]:
df_test.show(3)

+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|Survived|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| null|       Q|      -1|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| null|       S|      -1|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| null|       Q|      -1|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+--------+
only showing top 3 rows



In [23]:
df_tests = df_test.select('PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked')

In [24]:
df_tests.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        892|      -1|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| null|       Q|
|        893|      -1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| null|       S|
|        894|      -1|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| null|       Q|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 3 rows



In [25]:
df_train.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [26]:
df_concat = df_tests.union(df_train)


In [27]:
df_concat.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        892|      -1|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| null|       Q|
|        893|      -1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| null|       S|
|        894|      -1|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| null|       Q|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 3 rows



In [28]:
df_concat.count()

1309

In [29]:
type(df_concat)

pyspark.sql.dataframe.DataFrame

In [30]:
# Change the df_concat to pandas to make work easier

In [31]:
df_new = df_concat.toPandas()

In [32]:
type(df_new)

pandas.core.frame.DataFrame

In [33]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int32  
 1   Survived     1309 non-null   int32  
 2   Pclass       1309 non-null   int32  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int32  
 7   Parch        1309 non-null   int32  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(2), int32(5), object(5)
memory usage: 97.3+ KB


In [34]:
# Sorting on passengers

In [35]:
# Sort out female passengers in firstclass

In [36]:
df_femalefirst = df_concat.where((df_concat.Sex == 'female') & (df_concat.Pclass == 1))

In [37]:
print("{0} females out of {1} passengers are in Class1".format(df_femalefirst.count(), df_concat.count()))

144 females out of 1309 passengers are in Class1


In [38]:
type(df_femalefirst)

pyspark.sql.dataframe.DataFrame

In [39]:

df_femalefirst.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+---------------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|   Fare|          Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+---------------+--------+
|        904|      -1|     1|Snyder, Mrs. John...|female|23.0|    1|    0|      21228|82.2667|            B45|       S|
|        906|      -1|     1|Chaffee, Mrs. Her...|female|47.0|    1|    0|W.E.P. 5734| 61.175|            E31|       S|
|        914|      -1|     1|Flegenheim, Mrs. ...|female|null|    0|    0|   PC 17598|31.6833|           null|       S|
|        916|      -1|     1|Ryerson, Mrs. Art...|female|48.0|    1|    3|   PC 17608|262.375|B57 B59 B63 B66|       C|
|        918|      -1|     1|Ostby, Miss. Hele...|female|22.0|    0|    1|     113509|61.9792|            B36|       C|
+-----------+--------+------+-----------

In [40]:
from pyspark.sql.functions import desc

In [41]:
df_femalefirst1 = df_femalefirst.select(df_femalefirst.Name, df_femalefirst.Age).sort(df_femalefirst.Age.desc())

In [42]:
df_femalefirst1.show()

+--------------------+----+
|                Name| Age|
+--------------------+----+
|Cavendish, Mrs. T...|76.0|
|Crosby, Mrs. Edwa...|64.0|
|Compton, Mrs. Ale...|64.0|
|Andrews, Miss. Ko...|63.0|
|Straus, Mrs. Isid...|63.0|
|Stone, Mrs. Georg...|62.0|
|Warren, Mrs. Fran...|60.0|
|Bucknell, Mrs. Wi...|60.0|
|Fortune, Mrs. Mar...|60.0|
|Brown, Mrs. John ...|59.0|
|Lurette, Miss. Elise|58.0|
|Cardeza, Mrs. Jam...|58.0|
|Graham, Mrs. Will...|58.0|
|Bonnell, Miss. El...|58.0|
|Potter, Mrs. Thom...|56.0|
|White, Mrs. John ...|55.0|
|Lindstrom, Mrs. C...|55.0|
|Cornell, Mrs. Rob...|55.0|
|Rothschild, Mrs. ...|54.0|
|Eustis, Miss. Eli...|54.0|
+--------------------+----+
only showing top 20 rows



In [43]:
display(df_new.describe())

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,-0.058060,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.760044,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,-1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,-1.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [44]:
df_new.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,-1,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,-1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,-1,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
3,895,-1,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S
4,896,-1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S


In [45]:
df_concat.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [46]:
df_concat.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [47]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int32  
 1   Survived     1309 non-null   int32  
 2   Pclass       1309 non-null   int32  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int32  
 7   Parch        1309 non-null   int32  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(2), int32(5), object(5)
memory usage: 97.3+ KB


In [48]:
df_new.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked        True
dtype: bool

In [49]:
df_new.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,-0.058060,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.760044,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,-1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,-1.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [50]:
from pyspark.sql import functions as F
df_concat.select(F.mean("Age")).show()

+------------------+
|          avg(Age)|
+------------------+
|29.881137667304014|
+------------------+



In [51]:
df_new["Age"]

0       34.5
1       47.0
2       62.0
3       27.0
4       22.0
        ... 
1304    27.0
1305    19.0
1306     NaN
1307    26.0
1308    32.0
Name: Age, Length: 1309, dtype: float64

In [52]:
from pyspark.sql.functions import col

In [53]:
df_concat.groupBy("Embarked").count().orderBy(col("Embarked").desc()).show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  914|
|       Q|  123|
|       C|  270|
|    null|    2|
+--------+-----+



In [54]:
df_concat.groupBy("Sex").count().orderBy(col("Sex").desc()).show()

+------+-----+
|   Sex|count|
+------+-----+
|  male|  843|
|female|  466|
+------+-----+



In [55]:
display(df_concat.groupBy("Sex").count())

DataFrame[Sex: string, count: bigint]

In [56]:
df_concat.groupBy("Age").count().orderBy(col("Age").desc()).show()

+----+-----+
| Age|count|
+----+-----+
|80.0|    1|
|76.0|    1|
|74.0|    1|
|71.0|    2|
|70.5|    1|
|70.0|    2|
|67.0|    1|
|66.0|    1|
|65.0|    3|
|64.0|    5|
|63.0|    4|
|62.0|    5|
|61.0|    5|
|60.5|    1|
|60.0|    7|
|59.0|    3|
|58.0|    6|
|57.0|    5|
|56.0|    4|
|55.5|    1|
+----+-----+
only showing top 20 rows



In [57]:
from pyspark.sql import functions as F

In [58]:
df_concat.groupBy("Sex").agg(F.mean("Age")).show()

+------+-----------------+
|   Sex|         avg(Age)|
+------+-----------------+
|female|28.68708762886598|
|  male|30.58522796352584|
+------+-----------------+



In [59]:
df_concat.groupBy("Pclass","Embarked").agg(F.mean("Fare"), F.min("Fare"), F.max("Fare"), F.mean("Age")).show()

+------+--------+------------------+---------+---------+------------------+
|Pclass|Embarked|         avg(Fare)|min(Fare)|max(Fare)|          avg(Age)|
+------+--------+------------------+---------+---------+------------------+
|     3|       C| 11.02162376237624|   4.0125|  22.3583|21.448666666666668|
|     2|       C|23.300592857142856|     12.0|  41.5792|23.645833333333332|
|     1|       Q|              90.0|     90.0|     90.0|              38.0|
|     3|       Q|10.390820353982301|     6.75|   29.125|25.535714285714285|
|     2|       Q|11.735114285714287|   9.6875|    12.35|              49.0|
|     1|       C|106.84533049645388|  25.7417| 512.3292|           39.0625|
|     1|       S|  72.1480943502825|      0.0|    263.0| 39.12198675496689|
|     3|       S|14.435422064777342|      0.0|    69.55| 25.24706766917293|
|     2|       S|21.206921487603307|      0.0|     73.5|29.692887931034484|
|     1|    null|              80.0|     80.0|     80.0|              50.0|
+------+----

In [60]:
# To get the classification based on Pclass based on age we use crosstab

In [61]:
df_concat.stat.crosstab("Sex","Pclass").show()

+----------+---+---+---+
|Sex_Pclass|  1|  2|  3|
+----------+---+---+---+
|      male|179|171|493|
|    female|144|106|216|
+----------+---+---+---+



In [62]:
df_concat.groupBy("Sex").pivot("Pclass").mean("Age").show()

+------+------------------+------------------+------------------+
|   Sex|                 1|                 2|                 3|
+------+------------------+------------------+------------------+
|female|37.037593984962406|27.499223300970876|22.185328947368422|
|  male| 41.02927152317881|30.815379746835443|25.962263610315187|
+------+------------------+------------------+------------------+



In [63]:
# Data Munging

In [64]:
from pyspark.sql.functions import isnan,when,count,col
df_concat.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_concat.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|263|    0|    0|     0|   1| 1014|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [65]:
df_concat.where(df_concat.Fare.isNull()).show()

+-----------+--------+------+------------------+----+----+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|              Name| Sex| Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+------------------+----+----+-----+-----+------+----+-----+--------+
|       1044|      -1|     3|Storey, Mr. Thomas|male|60.5|    0|    0|  3701|null| null|       S|
+-----------+--------+------+------------------+----+----+-----+-----+------+----+-----+--------+



In [66]:
# Calculate the median fare for Pclass 3  and embarked at S

In [67]:
medianFare = df_concat.where((df_concat.Pclass == '3') & (df_concat.Embarked == "S")).approxQuantile("Fare",[0.5],0)

In [68]:
print(medianFare)

[8.05]


In [69]:
df_concat = df_concat.fillna(medianFare[0], ["Fare"])

In [70]:
df_concat.where(df_concat.Fare.isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [71]:
# Next we have 2 null values in 'Embarked'

In [72]:
df_concat.where(df_concat.Embarked.isNull()).show()

+-----------+--------+------+--------------------+------+----+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+----+-----+--------+
|         62|       1|     1| Icard, Miss. Amelie|female|38.0|    0|    0|113572|80.0|  B28|    null|
|        830|       1|     1|Stone, Mrs. Georg...|female|62.0|    0|    0|113572|80.0|  B28|    null|
+-----------+--------+------+--------------------+------+----+-----+-----+------+----+-----+--------+



In [73]:
# see where max people embarked on

In [74]:
df_concat.groupBy("Embarked").count().orderBy("count", ascending=False).show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  914|
|       C|  270|
|       Q|  123|
|    null|    2|
+--------+-----+



In [75]:
df_concat.where("Survived != -1").crosstab("Embarked","Survived").show()

+-----------------+---+---+
|Embarked_Survived|  0|  1|
+-----------------+---+---+
|             null|  0|  2|
|                Q| 47| 30|
|                S|427|217|
|                C| 75| 93|
+-----------------+---+---+



In [76]:
df_concat = df_concat.fillna({'Embarked' : 'S'})

In [77]:
df_concat.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_concat.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|263|    0|    0|     0|   0| 1014|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [78]:
# Now we will deal with the age. We will find the title from the name

In [79]:
from pyspark.sql.functions import udf
def getTitle(name:str):
    firstnamewithtitle = name.split(',')[1]
    title = firstnamewithtitle.split('.')[0]
    title = title.strip().lower()
    return title

getTitle_udf = udf(getTitle)

df_concat = df_concat.withColumn("Title", getTitle_udf("Name"))

In [80]:
df_concat.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|Title|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+-----+
|        892|      -1|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|   mr|
|        893|      -1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0| null|       S|  mrs|
|        894|      -1|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875| null|       Q|   mr|
|        895|      -1|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625| null|       S|   mr|
|        896|      -1|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875| null|       S|  mrs|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------

In [81]:
# Extract the title

In [82]:
df_concat.groupBy(df_concat.Title).count().show()

+------------+-----+
|       Title|count|
+------------+-----+
|         col|    4|
|          ms|    2|
|      master|   61|
|        miss|  260|
|          mr|  757|
|         mrs|  197|
|        dona|    1|
|         rev|    8|
|          dr|    8|
|         don|    1|
|    jonkheer|    1|
|the countess|    1|
|       major|    2|
|        capt|    1|
|        mlle|    2|
|        lady|    1|
|         sir|    1|
|         mme|    1|
+------------+-----+



In [83]:
# we are replacing every title with Mr, Mrs, Ms, Officer

In [84]:
df_concat = df_concat.replace(['mlle', 'mme','ms','major','lady','dona','don','the countess','jonkheer','capt',
                              'col','rev','sir'],['miss','mrs','miss','officer','mrs','mrs','mr','mrs','officer',
                                                  'officer','officer','officer','officer'])

In [85]:
df_concat.groupBy(df_concat.Title).count().show()

+-------+-----+
|  Title|count|
+-------+-----+
|officer|   17|
| master|   61|
|   miss|  264|
|     mr|  758|
|    mrs|  201|
|     dr|    8|
+-------+-----+



In [86]:
meanAgedf = df_concat.groupBy("Title").mean("Age").select("Title",col("avg(Age)"). alias("MeanAge"))

In [87]:
meanAgedf.show()

+-------+-----------------+
|  Title|          MeanAge|
+-------+-----------------+
|officer|47.05882352941177|
| master|5.482641509433963|
|   miss| 21.8243661971831|
|     mr|32.26546391752577|
|    mrs|36.97126436781609|
|     dr|43.57142857142857|
+-------+-----------------+



In [88]:
(df_concat.select("Title","Age").where(df_concat.Age.isNotNull())).show()

+-----+----+
|Title| Age|
+-----+----+
|   mr|34.5|
|  mrs|47.0|
|   mr|62.0|
|   mr|27.0|
|  mrs|22.0|
|   mr|14.0|
| miss|30.0|
|   mr|26.0|
|  mrs|18.0|
|   mr|21.0|
|   mr|46.0|
|  mrs|23.0|
|   mr|63.0|
|  mrs|47.0|
|  mrs|24.0|
|   mr|35.0|
|   mr|21.0|
| miss|27.0|
|  mrs|45.0|
|   mr|55.0|
+-----+----+
only showing top 20 rows



In [89]:
from pyspark.sql.functions import when,col
df_concat = df_concat.join(meanAgedf, "Title").withColumn("Age", when(isnan(col("Age")) | col("Age").isNull(), 
                                                                      col("MeanAge")). otherwise(col("Age"))).drop("MeanAge")

In [90]:
df_concat.show(3)

+-----+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|Title|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|   mr|        892|      -1|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| null|       Q|
|  mrs|        893|      -1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| null|       S|
|   mr|        894|      -1|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| null|       Q|
+-----+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 3 rows



In [91]:
df_concat.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_concat.columns]).show()

+-----+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|Title|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|    0|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0| 1014|       0|
+-----+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [92]:
df_concat = df_concat.drop("Cabin")

In [93]:
df_concat.show(3)

+-----+-----------+--------+------+--------------------+------+----+-----+-----+------+------+--------+
|Title|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Embarked|
+-----+-----------+--------+------+--------------------+------+----+-----+-----+------+------+--------+
|   mr|        892|      -1|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292|       Q|
|  mrs|        893|      -1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0|       S|
|   mr|        894|      -1|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875|       Q|
+-----+-----------+--------+------+--------------------+------+----+-----+-----+------+------+--------+
only showing top 3 rows



In [94]:
df_concat.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_concat.columns]).show()

+-----+-----------+--------+------+----+---+---+-----+-----+------+----+--------+
|Title|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Embarked|
+-----+-----------+--------+------+----+---+---+-----+-----+------+----+--------+
|    0|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|       0|
+-----+-----------+--------+------+----+---+---+-----+-----+------+----+--------+



In [95]:
df_new1 = df_concat.toPandas()

In [96]:
type(df_new1)

pandas.core.frame.DataFrame

In [97]:
df_new1.head()

,Title,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,mr,892,-1,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,mrs,893,-1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,mr,894,-1,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,mr,895,-1,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,mrs,896,-1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [98]:
df_new1.tail()

,Title,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
1304,officer,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.00,S
1305,miss,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.00,S
1306,miss,889,0,3,"""Johnston, Miss. Catherine Helen """"Carrie""""""",female,21.824366,1,2,W./C. 6607,23.45,S
1307,mr,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.00,C
1308,mr,891,0,3,"Dooley, Mr. Patrick",male,32.000000,0,0,370376,7.75,Q


In [99]:

df_new1Te = df_new1[df_new1.Survived == -1]

In [100]:
df_new1Te.tail()

,Title,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
413,mr,1305,-1,3,"Spector, Mr. Woolf",male,32.265464,0,0,A.5. 3236,8.0500,S
414,mrs,1306,-1,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C
415,mr,1307,-1,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,S
416,mr,1308,-1,3,"Ware, Mr. Frederick",male,32.265464,0,0,359309,8.0500,S
417,master,1309,-1,3,"Peter, Master. Michael J",male,5.482642,1,1,2668,22.3583,C


In [101]:
df_newTr = df_new1[df_new1.Survived != -1]

In [102]:
df_newTr.head()

,Title,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
418,mr,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
419,mrs,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
420,miss,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
421,mrs,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
422,mr,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [103]:
df_newTr.tail()

,Title,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
1304,officer,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.00,S
1305,miss,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.00,S
1306,miss,889,0,3,"""Johnston, Miss. Catherine Helen """"Carrie""""""",female,21.824366,1,2,W./C. 6607,23.45,S
1307,mr,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.00,C
1308,mr,891,0,3,"Dooley, Mr. Patrick",male,32.000000,0,0,370376,7.75,Q


In [104]:
# Logistic regression

In [106]:
X = df_newTr.iloc[:,[0,3,5,6,7,10,11]].values

In [107]:
X

array([['mr', 3, 'male', ..., 1, 7.25, 'S'],
       ['mrs', 1, 'female', ..., 1, 71.2833, 'C'],
       ['miss', 3, 'female', ..., 0, 7.925, 'S'],
       ...,
       ['miss', 3, 'female', ..., 1, 23.45, 'S'],
       ['mr', 1, 'male', ..., 0, 30.0, 'C'],
       ['mr', 3, 'male', ..., 0, 7.75, 'Q']], dtype=object)

In [108]:
y = df_newTr.iloc[:,2].values

In [109]:
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [110]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25, random_state=0)

In [111]:
from sklearn.preprocessing import StandardScaler
sc_X= StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

ValueError: could not convert string to float: 'mr'

In [112]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)

ValueError: could not convert string to float: 'mr'